In [ ]:
# Import libraries
import os
import requests
from zipfile import ZipFile
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from datetime import datetime
import time
import glob
import contextlib
from PIL import Image

# Main data source
# https://gbfs.beryl.cc/v2_2/Greater_Manchester/gbfs.json
# Updates every ~10 minutes

In [ ]:
# Create output folder
try:
    os.mkdir('glasgow-plots')
except:
    print('Output folder already exists')

In [16]:
# Set interval (minutes)
interval = 10

# Set runtime (minutes)
runtime = 60

# Calculate interval in seconds and number of iterations to perform
iterations = int(runtime/interval)
interval = interval*60

In [3]:
# Download map data (Approx. 65MB)
mapurl = 'http://download.geofabrik.de/europe/great-britain/england/greater-manchester-latest-free.shp.zip'
req = requests.get(mapurl, allow_redirects=True)
open('greater-manchester-latest-free.shp.zip', 'wb').write(req.content)

68718655

In [4]:
# Extract necessary files from downloaded zip file
with ZipFile('greater-manchester-latest-free.shp.zip', 'r') as zObject:
    zObject.extract('gis_osm_roads_free_1.cpg', path='greater-manchester-latest-free.shp')
    zObject.extract('gis_osm_roads_free_1.dbf', path='greater-manchester-latest-free.shp')
    zObject.extract('gis_osm_roads_free_1.prj', path='greater-manchester-latest-free.shp')
    zObject.extract('gis_osm_roads_free_1.shp', path='greater-manchester-latest-free.shp')
    zObject.extract('gis_osm_roads_free_1.shx', path='greater-manchester-latest-free.shp')
zObject.close()

In [5]:
# Request JSON for station information
stationsurl = 'https://gbfs.beryl.cc/v2_2/Greater_Manchester/station_information.json'
req = requests.get(stationsurl)
print(req)

# Extract stations from JSON
jsondata = json.loads(req.text)
stations = jsondata['data']['stations']

# Convert JSON to dataframe
data = []
for station in stations:
    stationdata = []
    stationdata.append(station['station_id'])
    stationdata.append(station['name'])
    stationdata.append(station['lat'])
    stationdata.append(station['lon'])
    stationdata.append(station['capacity'])
    stationdata.append(station['rental_uris'])
    data.append(stationdata)
    
data = pd.DataFrame(data, columns=['station_id', 'name', 'lat', 'lon', 'capacity', 'rental_uris'])

# Convert to geodataframe
geometry = [Point(xy) for xy in zip(data['lon'], data['lat'])]
data = gpd.GeoDataFrame(data, crs='epsg:4326', geometry=geometry)
data.head()

<Response [200]>


,station_id,name,lat,lon,capacity,rental_uris,geometry
0,4140,Beetham Tower,53.475572,-2.251275,10,{'android': 'https://beryl.app/gbfs?$deeplink_...,POINT (-2.25128 53.47557)
1,3001,University Shopping Centre,53.467239,-2.235462,14,{'android': 'https://beryl.app/gbfs?$deeplink_...,POINT (-2.23546 53.46724)
2,4042,Frederick Street,53.484803,-2.256625,4,{'android': 'https://beryl.app/gbfs?$deeplink_...,POINT (-2.25663 53.48480)
3,4043,Camp Street/Broughton Lane,53.497120,-2.264402,8,{'android': 'https://beryl.app/gbfs?$deeplink_...,POINT (-2.26440 53.49712)
4,4044,Trafford College,53.456848,-2.290090,13,{'android': 'https://beryl.app/gbfs?$deeplink_...,POINT (-2.29009 53.45685)


In [6]:
# Read roads data
roads = gpd.read_file('greater-manchester-latest-free.shp/gis_osm_roads_free_1.shp')

# Filter minor roads
roads = roads[~roads['fclass'].isin(['residential', 'footway', 'cycleway', 'bridleway', 'busway',
                                     'track_grade1', 'track_grade2', 'track_grade3', 'track_grade4',
                                     'track_grade5', 'track_grade6', 'path', 'track', 'steps', 'unknown',
                                     'service'])]
roads.head()

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,718463,5112,trunk,Devonshire Street,A665,B,48,0,F,F,"LINESTRING (-2.21571 53.46883, -2.21581 53.468..."
5,780214,5121,unclassified,Dover Street,None,B,32,0,F,F,"LINESTRING (-2.23160 53.46527, -2.23155 53.465..."
8,780336,5115,tertiary,Grafton Street,None,B,48,0,F,F,"LINESTRING (-2.23014 53.46295, -2.23002 53.462..."
16,783632,5124,pedestrian,None,None,B,0,0,F,F,"LINESTRING (-2.23596 53.46902, -2.23590 53.469..."
17,783634,5115,tertiary,Booth Street East,None,F,48,0,F,F,"LINESTRING (-2.23541 53.46854, -2.23469 53.468..."


In [11]:
# Function to fetch and plot data
def fetchandplot(roads, data):
    # Request JSON
    liveurl = 'https://gbfs.beryl.cc/v2_2/Greater_Manchester/station_status.json'
    req = requests.get(liveurl)

    # Generate filename from timestamp
    now = datetime.now()
    filename = 'manchester-plots/' + now.strftime("%Y%m%d %H%M%S") + '.png'

    # Extract stations from JSON
    jsondata = json.loads(req.text)
    stations = jsondata['data']['stations']

    # Convert JSON to dataframe
    livedata = []
    for station in stations:
        stationdata = []
        stationdata.append(station['station_id'])
        stationdata.append(station['num_bikes_available'])
        stationdata.append(station['num_docks_available'])
        stationdata.append(station['is_installed'])
        stationdata.append(station['is_renting'])
        stationdata.append(station['last_reported'])
        livedata.append(stationdata)

    livedata = pd.DataFrame(livedata, columns=['station_id', 'num_bikes_available', 'num_docks_available', 
                                               'is_installed', 'is_renting', 'last_reported'])
    livedata = livedata[['station_id', 'num_bikes_available', 'num_docks_available']]

    # Merge livedata with stationsdata, calculate fill percent for each station
    livedata = livedata.merge(data, on='station_id')
    livedata['fillpercent'] = livedata['num_bikes_available']/livedata['num_docks_available']*100
    livedata['fillpercent'] = livedata['fillpercent'].clip(upper=100)

    # Plot station fill percentages, save image
    fig, ax = plt.subplots(figsize=(12, 8))
    roads.plot(ax=ax, alpha=0.3, color='grey')
    data.plot(column=livedata['fillpercent'], cmap='coolwarm_r', markersize=100, ax=ax, legend=True)
    ax.set_xlim(-2.32, -2.2)
    ax.set_ylim(53.435, 53.505)
    ax.set_xticks([])
    ax.set_yticks([])
    plt.title('Beryl Bikes Available in Manchester (as % of Station Capacity). ' + now.strftime('Time: %H:%M'))
    fig.savefig(filename, dpi=400, bbox_inches='tight')
    plt.close()

In [12]:
# Test working before running main loop
fetchandplot(roads, data)

In [ ]:
# Generate frames based on provided interval and runtime
frames = 0
while frames < iterations:
    fetchandplot(roads, data)
    frames += 1
    time.sleep(interval - time.time() % interval)

In [14]:
# Filepaths
fp_in = 'manchester-plots/*.png'
fp_out = 'manchester-plots/manchester-bike-movements.gif'

# Use ExitStack to automatically close opened images
with contextlib.ExitStack() as stack:
    # Load images
    images = (stack.enter_context(Image.open(f)) for f in sorted(glob.glob(fp_in)))
    # Extract first image from iterator
    image = next(images)
    # Save as gif
    image.save(fp=fp_out, format='gif', append_images=images, save_all=True, duration=100, loop=0)